<a href="https://colab.research.google.com/github/rtajeong/ChatGPT_for_Management/blob/main/2_gpt_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT (Generative Pre-trained Transformer)
- 정의: 텍스트 데이터를 학습하여 자연어를 생성하고 이해할 수 있는 인공지능 모델
- 훈련 데이터는 인터넷에서 크롤링한 텍스트(예: Wikipedia, 책, 웹 문서 등)와 추가적으로 대화 스타일 데이터로 구성
- 주요 기능:
  - 질문에 답변
  - 텍스트 생성 (예: 이메일 작성, 보고서 작성)
  - 번역
  - 요약
  - 데이터 분석 보조, 등
- 동작 방식:
  - Transformer 구조 (decoder 부분)
    - 입력 텍스트 처리: GPT는 단어를 숫자로 변환해 이해(예: "Hello" → 숫자 배열).
    - Attention 메커니즘: 단어 간의 관계를 이해. 예를 들어, "The bank"라는 문장에서 "bank"가 물리적 은행인지 강변(bank)인지 문맥을 통해 구별.
    - 출력 생성: 입력 데이터를 기반으로 가장 적합한 단어를 예측해 텍스트를 생성.
  - 사전 학습 (Pre-training)과 미세 조정 (Fine-tuning):
    - 사전 학습: 인터넷의 방대한 데이터를 읽고 언어 패턴을 학습.
    - 미세 조정: 특정 작업(예: 금융 데이터 분석)에 맞게 모델을 추가 학습


# 인공지능 모델(e.g. GPT) 훈련 과정
- 사전 훈련(Pre-training):
  - GPT는 방대한 인터넷 데이터를 사용하여 언어 패턴을 학습
  - 목표: 문맥을 이해하고 다음 단어를 예측하는 능력을 학습.
  - 예: "The stock market is" → "rising" (다음 단어를 예측)
- 미세 조정(Fine-tuning):
  - 특정 업무(예: 고객 이메일 작성)에 맞게 모델을 추가 학습.
  - 목표: 사전 훈련된 모델을 원하는 업무에 최적화.
  - 예: "Write a polite apology for delayed shipping" → 완성된 이메일 초안 작성.

## 간단한 훈련 예시 1 (단어 예측)
- 훈련 과정을 학생들이 직접 경험할 수 있도록 텍스트 데이터를 생성하고, 미니 모델을 훈련시키는 간단한 과정을 보인다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 훈련 데이터 (간단한 텍스트)
data = "The stock market is unpredictable but has trends. Stock trading requires analysis."

# 1. 토큰화 및 데이터 전처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])  # 단어를 추출하고 고유한 인덱스를 부여 (index 1 부터 부여)
                                # 인덱스 0 은 padding 용으로 예약되어 있음
sequence = tokenizer.texts_to_sequences([data])[0]
print(sequence)
print(tokenizer.word_index)

[2, 1, 3, 4, 5, 6, 7, 8, 1, 9, 10, 11]
{'stock': 1, 'the': 2, 'market': 3, 'is': 4, 'unpredictable': 5, 'but': 6, 'has': 7, 'trends': 8, 'trading': 9, 'requires': 10, 'analysis': 11}


In [ ]:
vocab_size = len(tokenizer.word_index) + 1
sequences = []
for i in range(1, len(sequence)):
    sequences.append(sequence[:i+1])

sequences

[[2, 1],
 [2, 1, 3],
 [2, 1, 3, 4],
 [2, 1, 3, 4, 5],
 [2, 1, 3, 4, 5, 6],
 [2, 1, 3, 4, 5, 6, 7],
 [2, 1, 3, 4, 5, 6, 7, 8],
 [2, 1, 3, 4, 5, 6, 7, 8, 1],
 [2, 1, 3, 4, 5, 6, 7, 8, 1, 9],
 [2, 1, 3, 4, 5, 6, 7, 8, 1, 9, 10],
 [2, 1, 3, 4, 5, 6, 7, 8, 1, 9, 10, 11]]

In [ ]:
# 입력(X)과 출력(Y) 분리
X, Y = [], []
for seq in sequences:
    X.append(seq[:-1])
    Y.append(seq[-1])
X = tf.keras.preprocessing.sequence.pad_sequences(X)
Y = to_categorical(Y, num_classes=vocab_size)

for i in range(len(X)):
    print("X: {}, Y: {} ({})".format(X[i], Y[i], tokenizer.index_word[i+1]))  # index assigned to each word starts from 1, not 0


X: [0 0 0 0 0 0 0 0 0 0 2], Y: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] (stock)
X: [0 0 0 0 0 0 0 0 0 2 1], Y: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] (the)
X: [0 0 0 0 0 0 0 0 2 1 3], Y: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] (market)
X: [0 0 0 0 0 0 0 2 1 3 4], Y: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] (is)
X: [0 0 0 0 0 0 2 1 3 4 5], Y: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] (unpredictable)
X: [0 0 0 0 0 2 1 3 4 5 6], Y: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] (but)
X: [0 0 0 0 2 1 3 4 5 6 7], Y: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] (has)
X: [0 0 0 2 1 3 4 5 6 7 8], Y: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] (trends)
X: [0 0 2 1 3 4 5 6 7 8 1], Y: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] (trading)
X: [0 2 1 3 4 5 6 7 8 1 9], Y: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] (requires)
X: [ 2  1  3  4  5  6  7  8  1  9 10], Y: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] (analysis)


In [ ]:
# 2. 모델 생성
model = Sequential([
    Embedding(vocab_size, 10, input_length=X.shape[1]),
    LSTM(50, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [ ]:
# 훈련 전에는 랜덤 단어 출력
print("다음 단어 예측(훈련 전): 'The stock market is' →", predict_next_word(model, tokenizer, "The stock market is"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
다음 단어 예측(훈련 전): 'The stock market is' → the


In [ ]:
# 3. 모델 훈련
model.fit(X, Y, epochs=100, verbose=0)

# 4. 결과 확인 (다음 단어 예측)
def predict_next_word(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])[0]
    sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=X.shape[1])
    predicted = model.predict(sequence)
    predicted_word = tokenizer.index_word[tf.argmax(predicted, axis=1).numpy()[0]]
    return predicted_word

print("다음 단어 예측(훈련 후): 'The stock market is' →", predict_next_word(model, tokenizer, "The stock market is"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
다음 단어 예측(훈련 후): 'The stock market is' → unpredictable


In [ ]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (1, 11, 10)                 │             120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (1, 50)                     │          12,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (1, 12)                     │             612 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,798 (151.56 KB)

 Trainable params: 12,932 (50.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,866 (101.04 KB)

## 간단한 훈련 예시 2 (문장 생성)
- GPT가 어떻게 텍스트를 생성하는지를 간단히 이해시키기 위함 (동작 원리)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

# 1. 훈련 데이터 정의
data = "The stock market is unpredictable but has trends. Stock trading requires careful analysis."

# 2. 토큰화 및 데이터 전처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
sequence = tokenizer.texts_to_sequences([data])[0]

vocab_size = len(tokenizer.word_index) + 1
sequences = []
for i in range(1, len(sequence)):
    sequences.append(sequence[:i+1])

# 입력(X)과 출력(Y) 분리
X, Y = [], []
for seq in sequences:
    X.append(seq[:-1])
    Y.append(seq[-1])
X = tf.keras.preprocessing.sequence.pad_sequences(X)
Y = to_categorical(Y, num_classes=vocab_size)

# 3. 간단한 모델 정의
model = Sequential([
    Embedding(vocab_size, 10, input_length=X.shape[1]),
    LSTM(50, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy')

# 4. 모델 학습
model.fit(X, Y, epochs=200, verbose=0)

# 5. 문장 생성 함수
def generate_text(model, tokenizer, seed_text, next_words=10):
    for _ in range(next_words):
        sequence = tokenizer.texts_to_sequences([seed_text])[0]
        sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=X.shape[1])
        predicted = model.predict(sequence)
        predicted_word = tokenizer.index_word[np.argmax(predicted)]
        seed_text += " " + predicted_word
    return seed_text

# 6. 결과 확인
seed_text = "The stock market"
generated_text = generate_text(model, tokenizer, seed_text, next_words=10)
print("Generated Text:")
print(generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Generated Text:
The stock market is unpredictable unpredictable has trends stock trading requires careful analysis


- 위의 예제를 통해 알 수 있는 것은 다음과 같다.
-  언어모델의 원리:
  - 초기 텍스트를 기반으로 문맥에 맞는 단어를 하나씩 예측하여 문장을 확장.
- 모델의 한계:
  - 단순 데이터로 학습했기 때문에 복잡한 문장 생성이 어려움.
  - GPT는 방대한 데이터로 학습해 이 과정을 훨씬 더 정교하게 수행.

# GPT구조
- Transformer 의 Decoder Stack
  - Multi-Head Self-Attention
  - Position-Wise Feedforward Network
  - Residual Connection 및 Layer Normalization
  - Masking: 순방향 정보를 가리기 위해 사용.
- 실습
  - 작은 크기의 GPT 모델을 구성하고 Attention Score를 시각화

In [ ]:
from transformers import GPT2Model, GPT2Tokenizer
import torch

# 모델 및 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

# Hidden states 활성화
model.config.output_hidden_states = True

# 입력 처리
input_text = "ChatGPT is an AI model"
inputs = tokenizer(input_text, return_tensors="pt")
print(inputs)

# 모델 실행
# outputs = model(**inputs)
outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])

# Hidden states 출력
hidden_states = outputs.hidden_states
print(f"Number of layers: {len(hidden_states)}")  # 모든 레이어 출력 개수 확인
print(f"Shape of one layer: {hidden_states[0].shape}")  # 각 레이어의 텐서 크기

print(outputs.last_hidden_state.shape)  # 마지막 레이어의 출력 벡터


{'input_ids': tensor([[30820,    38, 11571,   318,   281,  9552,  2746]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Number of layers: 13
Shape of one layer: torch.Size([1, 7, 768])
torch.Size([1, 7, 768])


In [ ]:
inputs

{'input_ids': tensor([[30820,    38, 11571,   318,   281,  9552,  2746]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

- attention_mask의 의미
  - 동일한 길이의 시퀀스를 처리하도록 패딩된 입력 데이터 위치는 정보가 없는 자리이므로, 모델이 이를 학습하거나 처리하지 않도록 마스킹한다. (1: 해당 위치의 토큰 처리), 0: 해당 위치의 토큰을 무시(마스킹))
  - 순방향 정보만을 처리하도록 추가적으로 Causal Masking도 사용 (왼쪽에서 오른쪽으로 순차적인 예측을 수행)

In [ ]:
print(tokenizer.decode(inputs['input_ids'][0]))
[tokenizer.decode(i) for i in inputs['input_ids'][0]]

ChatGPT is an AI model


['Chat', 'G', 'PT', ' is', ' an', ' AI', ' model']

In [ ]:
[s.shape for s in outputs.hidden_states]  # 모든 hidden state 들의 shape (batch_size, sequence_length, hidden_size)
                                          # hidden_size 는 embedding vector size 임.

[torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768]),
 torch.Size([1, 7, 768])]

- hidden_states는 기본적으로 반환되지 않는다.
- 반환하려면 model.config.output_hidden_states = True 또는 output_hidden_states=True 옵션을 활성화.

In [ ]:
print(outputs.last_hidden_state.shape)

torch.Size([1, 7, 768])


- outputs.last_hidden_state.shape:
  - (1, sequence_length, hidden_size)
  - sequence_length는 입력 토큰의 길이, hidden_size는 모델의 차원 크기(예: GPT2의 경우 768)

In [ ]:
model.config

GPT2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "output_hidden_states": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.47.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [ ]:
print(model)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)


In [ ]:
for name, module in model.named_modules():
    print(f"Layer: {name}, Type: {type(module)}")

Layer: , Type: <class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
Layer: wte, Type: <class 'torch.nn.modules.sparse.Embedding'>
Layer: wpe, Type: <class 'torch.nn.modules.sparse.Embedding'>
Layer: drop, Type: <class 'torch.nn.modules.dropout.Dropout'>
Layer: h, Type: <class 'torch.nn.modules.container.ModuleList'>
Layer: h.0, Type: <class 'transformers.models.gpt2.modeling_gpt2.GPT2Block'>
Layer: h.0.ln_1, Type: <class 'torch.nn.modules.normalization.LayerNorm'>
Layer: h.0.attn, Type: <class 'transformers.models.gpt2.modeling_gpt2.GPT2SdpaAttention'>
Layer: h.0.attn.c_attn, Type: <class 'transformers.pytorch_utils.Conv1D'>
Layer: h.0.attn.c_proj, Type: <class 'transformers.pytorch_utils.Conv1D'>
Layer: h.0.attn.attn_dropout, Type: <class 'torch.nn.modules.dropout.Dropout'>
Layer: h.0.attn.resid_dropout, Type: <class 'torch.nn.modules.dropout.Dropout'>
Layer: h.0.ln_2, Type: <class 'torch.nn.modules.normalization.LayerNorm'>
Layer: h.0.mlp, Type: <class 'transformers.models.gpt2

- 모델 시각화

In [ ]:
!pip install torchviz

In [ ]:
from torchviz import make_dot

inputs = tokenizer("ChatGPT is amazing", return_tensors="pt")
outputs = model(**inputs)

make_dot(outputs.last_hidden_state, params=dict(model.named_parameters())).render("model_structure", format="png")


'model_structure.png'

# Exercise



- 토큰 생성

In [ ]:
from transformers import GPT2Tokenizer

texts = ["Hello, how are you? I am fine, thank you!"]
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenized_texts = [tokenizer.encode(text) for text in texts]
print(tokenized_texts)


[[15496, 11, 703, 389, 345, 30, 314, 716, 3734, 11, 5875, 345, 0]]


In [ ]:
print(tokenizer.decode(tokenized_texts[0]))
[tokenizer.decode(i) for i in tokenized_texts[0]]

Hello, how are you? I am fine, thank you!


['Hello',
 ',',
 ' how',
 ' are',
 ' you',
 '?',
 ' I',
 ' am',
 ' fine',
 ',',
 ' thank',
 ' you',
 '!']